# Finetune ruGPT3Large on SRE data


## Install enviroment

In [ ]:
!pip3 install urllib3

In [ ]:
!pip3 install pandas

In [ ]:
!pip3 install tensorboard

In [42]:
!pip install transformers==3.0.2

Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.24.0
    Uninstalling transformers-4.24.0:
      Successfully uninstalled transformers-4.24.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 2.2.2 requires transformers<5.0.0,>=4.6.0, but you have transformers 3.0.2 which is incompatible.


In [ ]:
ls ~/tmp

In [ ]:
pip uninstall torch -y

In [ ]:
cat /usr/local/cuda/version.txt

In [ ]:
import torch
print(torch.__version__)

In [ ]:
!TMPDIR=~/tmp pip install torch==1.8.1+cu101 torchvision==0.9.1+cu101 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
import torch
print(torch.__version__)

In [ ]:
torch.cuda.is_available()

In [ ]:
!python -m wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

##Apex setup

Clone data, move apex/apex out of the apex folder

In [ ]:
!git clone -b 22.04-dev https://github.com/NVIDIA/apex

In [ ]:
!cd apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

In [ ]:
!ls 

## Prepare data

In [1]:
!ls

'Finetune SRE GPT3Large.ipynb'
'GPT_models_evaluation_n_shots (4).ipynb'
 SRE-15
 apex
 apex_
 aste_pipeline.ipynb
 examples
 ghostdriver.log
 gpt2_cached_lm_1024_train.txt
 pretrain_transformers.py
 rugpt3l
 rugpt3large_based_on_gpt2_finetuned-50x3
 rugpt3small_based_on_gpt2_finetuned-50x3
 runs
 train.txt
 valid.txt


In [2]:
data_path_car = "~/ASTE/SRE-15/SentiRuEval_car_train_short_edited.csv"
data_path_rest = "~/ASTE/SRE-15/SentiRuEval_rest_train_short_edited.csv"
data_path_film = "~/ASTE/SRE-15/SentiRuEval_film_train_short_edited.csv"

In [22]:
import pandas as pd
data_car = pd.read_csv(data_path_car)
data_rest = pd.read_csv(data_path_rest)
data_film = pd.read_csv(data_path_film)

In [30]:
def format_columns(text, response):
  return '<s>Текст: ' + text + '\n' + response.strip()+'</s>'

In [31]:
texts_car = list(map(format_columns, data_car["Text"], data_car["Response"]))
texts_rest = list(map(format_columns, data_rest["Text"], data_rest["Response"]))
texts_film = list(map(format_columns, data_film["Text"], data_film["Response"]))

In [32]:
print(texts_car[0])
#print(texts_rest[0])

<s>Текст: Недавно купил этот автомобиль. Авто отличное! Двигатель 2,5 литра, турбодизель. Прежний хозяин сказал при продаже, что масло не жрет, солярку тоже, летит как угорелая! Так оно и есть. 140 км/ч нормальная крейсерская скорость. Вообще немцы умеют делать автомобили. Дорогу держит отлично, так как достаточно широкая машина. Тормоза все дисковые. Главное передний привод, по сравнению с другими немецкими автомобилями. Такими как мерседес и бмв этого же класса. Места в автомобиле очень много. Не тесно даже, когда сидят пять взрослых человек. Багажное отделение тоже огромно. Влезла стиральная машина. По соотношению цена - качество, отличный автомобиль. Больше никогда не сяду за руль российского автомбиля! Всем рекомендую Ауди. 
Авто: отличное: положительно 
двигатель: 2,5 литра, турбодизель: положительно 
прежний хозяин: сказал при продаже, что масло не жрет, солярку тоже, летит как угорелая: положительно 
крейсерская скорость: 140 км/ч: положительно 
дорогу держит: отлично: положите

We use the first 50 texts as train, other data as val

In [33]:
split = 50
train = texts_car[:split] + texts_rest[:split] + texts_film[:split]
valid = texts_car[split:] + texts_rest[split:] + texts_film[split:]

In [34]:
len(train), len(valid)

(150, 132)

In [35]:
with open("train.txt", "w") as file:
    file.write("\n\n".join(train))

In [36]:
with open("valid.txt", "w") as file:
    file.write("\n\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [15]:
import torch
torch.cuda.empty_cache()

In [19]:
!nvidia-smi

Wed Dec 14 13:41:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.126.02   Driver Version: 418.126.02   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   37C    P0    44W / 300W |     11MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [20]:
!python pretrain_transformers.py \
    --output_dir=rugpt3l\
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3large_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --fp16 \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 6 \
    --block_size 2048 \
    --overwrite_output_dir\
    --block_by_block_with_newline

12/14/2022 13:41:47 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
12/14/2022 13:41:48 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt3large_based_on_gpt2/config.json from cache at /data/home/asurikov/.cache/torch/transformers/53218293a9edec913332b4f2d178496a60f98d64a1af74f92984804152f9404c.02a103afdbdbf4896cc41fc6495e47b7e5e2f353a287fe98d178e669be028903
12/14/2022 13:41:48 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 1536,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
 

## Check our model (load in)

In [20]:
checkpoint_path = "rugpt3large_based_on_gpt2_finetuned-50+50"

from transformers import GPT2LMHeadModel, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint_path)
model = GPT2LMHeadModel.from_pretrained(checkpoint_path)

In [21]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1536)
    (wpe): Embedding(2048, 1536)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [ ]:
!nvidia-smi

In [22]:
print('Текст: ' + data_car.iloc[73]['Text'].strip() + '\n')

Текст: Большой и вместительный автомобиль. Едет мягко и в тоже время не покачивает как на корабле. Места для задних пассажиров очень предостаточно и как показывает личный опыт, с весом 100 с лишним килограмм и ростом за 180 сантиметров прекрасно умещаемся. Едешь и разуешься, шум в салоне практически отсутствует, красота, можно говорить не напрягаясь, все хорошо слышно. Проехав не одну тысячу километров по бездорожью понимаешь полезность полного привода, ни разу не подвел. Кому не ездить по лесам и карьерам, можно попробовать и просто переднеприводный, но только по трассе, так как сам по себе автомобиль тяжелый.



In [23]:
from textwrap import wrap

prompt = 'Текст: ' + data_car.iloc[73]['Text'].strip() + '\n'
input_ids = tokenizer.encode(prompt, return_tensors="pt")
input_ids = input_ids.cuda()
out = model.generate(
        input_ids, 
        min_length=100, 
        max_length=400, 
        eos_token_id=5, 
        pad_token_id=1,
        top_k=5,
        top_p=0.0,
        no_repeat_ngram_size=5
)
generated_text = list(map(tokenizer.decode, out))[0]
print('\n' + '\n'.join(wrap(prompt, 100)) + '\n')
print(generated_text[len(prompt):])
#print(generated_text[len(prompt):].split("\n"))


Текст: Большой и вместительный автомобиль. Едет мягко и в тоже время не покачивает как на корабле.
Места для задних пассажиров очень предостаточно и как показывает личный опыт, с весом 100 с лишним
килограмм и ростом за 180 сантиметров прекрасно умещаемся. Едешь и разуешься, шум в салоне
практически отсутствует, красота, можно говорить не напрягаясь, все хорошо слышно. Проехав не одну
тысячу километров по бездорожью понимаешь полезность полного привода, ни разу не подвел. Кому не
ездить по лесам и карьерам, можно попробовать и просто переднеприводный, но только по трассе, так
как сам по себе автомобиль тяжелый.

Машина: надежная, неприхотливая: положительно 
Машина: быстрая: положительно 
Автомобиль: просторный: положительно 
Езда: мягкая: положительно 
Места: для задних пассажиров очень много: положительно 
Шум: практически отсутствует: положительно 
Проезд: не одну тысячу километров: положительно

<pad><pad><pad><pad><pad> размещения<pad><pad><pad><pad> финала работа: положительно 


In [ ]:
del model

## Check our model (interctive script)

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=comment_model \
    --k=5 \
    --p=0.95 \
    --length=50

Context >>> Is it

ruGPT:
Is it's the case that #Microsoft is using #SAP to manage #Microsoft’s business?

Context >>> It  is

ruGPT:
It is the only one that I've been trying to do.
Siri is on the verge of death, and I'm not aware of why she has to be on the verge.

Context >>> Why

ruGPT:
Why do I go here?  I'm wondering if I can use the #Microsoft Windows 8 Pro for the #WindowsPhone phone?  Thanks! 